# Personalized User-level DP

In [1]:
from autodp import rdp_acct
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools

import os
path_project = os.path.dirname(os.path.abspath('.'))
import sys
sys.path.append(os.path.join(path_project, 'src'))
sys.path.append(os.path.join(path_project, 'exp/script'))

import options

img_path = os.path.join(path_project, 'exp', 'img')
pickle_path = os.path.join(path_project, 'exp', 'pickle')
results_path = os.path.join(path_project, 'exp', 'results')
default_args = options.build_default_args(path_project)

import copy
from run_simulation import run_simulation

from mylogger import logger_set_debug, logger_set_info, logger_set_warning
from opacus.accountants import RDPAccountant

import pickle

def RDP_gaussian_with_C(sigma, alpha, C):
    assert(sigma > 0)
    assert(alpha >= 0)
    return 0.5 * C**2 / sigma ** 2 * alpha

### utils

In [2]:
Q_LIST_SIZE = 30

# binary search given q_u
def from_q_u(q_u, delta, epsilon_u, sigma, T, m=100, precision=1e-6):
    max_sensitivity_u = 100
    min_sensitivity_u = 0
    while True:
        sensitivity_u = (max_sensitivity_u + min_sensitivity_u) / 2
        # func_gaussian = lambda x: RDP_gaussian_with_C(sigma, x, sensitivity_u)
        # accountant = rdp_acct.anaRDPacct(m=m)
        accountant = RDPAccountant()
        for i in range(T):
            accountant.step(noise_multiplier=sigma/sensitivity_u, sample_rate=q_u)
            # accountant.compose_subsampled_mechanisms_lowerbound(func=func_gaussian, prob=q_u)
        # eps = accountant.get_eps(delta)
        eps = accountant.get_epsilon(delta=delta)
        if eps < epsilon_u:
            min_sensitivity_u = sensitivity_u
        else:
            max_sensitivity_u = sensitivity_u
        if 0 < epsilon_u - eps and epsilon_u - eps < precision:
            return sensitivity_u, eps


# binary search given_sensitivity_u
def from_sensitivity_u(sensitivity_u, delta, epsilon_u, sigma, T, m=100, precision=1e-6):
    max_q_u = 1.0
    min_q_u = 0
    # func_gaussian = lambda x: RDP_gaussian_with_C(sigma, x, sensitivity_u)
    while True:
        q_u = (max_q_u + min_q_u) / 2
        # accountant = rdp_acct.anaRDPacct(m=m)
        accountant = RDPAccountant()
        for i in range(T):
            # accountant.compose_subsampled_mechanisms_lowerbound(func=func_gaussian, prob=q_u)
            accountant.step(noise_multiplier=sigma/sensitivity_u, sample_rate=q_u)
        # eps = accountant.get_eps(delta)
        eps = accountant.get_epsilon(delta=delta)
        if eps < epsilon_u:
            min_q_u = q_u
        else:
            max_q_u = q_u
        if 0 < epsilon_u - eps and epsilon_u - eps < precision:
            return q_u, eps
        

# qCカーブを書くために，適当にqを選んでCを計算して点をプロットする
def make_q_c_curve(epsilon_u, delta, sigma, n_round=100, num_points=20, min=-5):
    T = n_round

    num_points = num_points // 3 * 2
    x = np.logspace(min, -1, num_points).tolist() + np.linspace(0.15, 1.0, int(num_points/2)).tolist()
    y = []
    for q_u in x:
        sensitivity_u, eps = from_q_u(q_u=q_u, delta=delta, epsilon_u=epsilon_u, sigma=sigma, T=T)
        assert eps <= epsilon_u, f"eps={eps} > epsilon_u={epsilon_u}"
        # print("sensitivity_u =", sensitivity_u, "eps =", eps)
        y.append(sensitivity_u)
    return x, y


def plot_q_c_curve(x, y, title="", log=True):
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_subplot(111)
    ax.plot(x, y, marker='o', label='sensitivity_u')
    for i, (xi, yi) in enumerate(zip(x, y)):
        if i % 5 == 0:
            ax.annotate(f"({xi:.5f}, {yi:.5f})", (xi, yi), textcoords="offset points", xytext=(0,10), ha='center')
    # q*C の値をプロット
    ax.plot(x, np.array(x)*np.array(y), marker='x', linestyle='--', color='red', label='product_x*y')
    print("Max idx", np.argmax(np.array(x)*np.array(y)))
    if log:
        ax.set_xscale("log")
        ax.set_yscale("log")
    ax.set_xlabel("q_u")
    ax.set_ylabel("sensitivity_u")
    ax.set_title(title)
    plt.show()


def make_static_params(epsilon_u_dct, delta, sigma, n_round, idx_per_group, q_step_size):
    """
    Need to specify idx and q_step_size
    """
    C_u_dct = {}
    q_u_dct = {}

    C_and_q_per_group = {}
    for group_eps, idx in idx_per_group.items():
        n_of_q_u = Q_LIST_SIZE
        q_u_list = []
        init_q_u = 1.0
        for _ in range(n_of_q_u):
            q_u_list.append(init_q_u)
            init_q_u *= q_step_size
        q_u = q_u_list[idx]
        C_u, _eps = from_q_u(q_u=q_u, delta=delta, epsilon_u=group_eps, sigma=sigma, T=n_round)
        assert _eps <= group_eps, f"_eps={_eps} > eps_u={group_eps}"
        C_and_q_per_group[group_eps] = (C_u, q_u)

    for user_id, eps_u in epsilon_u_dct.items():
        C_u, q_u = C_and_q_per_group[eps_u]
        C_u_dct[user_id] = C_u
        q_u_dct[user_id] = q_u

    return C_u_dct, q_u_dct


def fed_simulation(
    delta, 
    sigma, 
    n_users, 
    C_u=None, 
    q_u=None, 
    q_step_size=None,
    times=1, 
    user_dist="uniform-iid", 
    silo_dist="uniform", 
    dataset_name="light_mnist", 
    global_learning_rate=10.0, 
    clipping_bound=1.0,
    n_round=10, 
    local_epochs=50, 
    local_learning_rate=0.01,
    agg_strategy="PULDP-AVG",
    epsilon_u=None,
    group_thresholds=None,
    validation_ratio=0.0,
):
    args = options.build_default_args(path_project)

    if dataset_name == "heart_disease":
        from flamby_utils.heart_disease import update_args

        args = update_args(args)

    elif dataset_name == "tcga_brca":
        from flamby_utils.tcga_brca import update_args

        args = update_args(args)

    args.dataset_name = dataset_name
    args.agg_strategy = agg_strategy
    args.n_total_round = n_round
    args.n_users = n_users
    args.local_epochs = local_epochs
    args.times = times

    args.user_dist = user_dist
    args.silo_dist = silo_dist
    args.global_learning_rate = global_learning_rate
    args.local_learning_rate = local_learning_rate
    args.clipping_bound = clipping_bound

    args.delta = delta
    args.sigma = sigma
    args.C_u = C_u
    args.q_u = q_u
    args.q_step_size = q_step_size
    args.epsilon_u = epsilon_u
    args.group_thresholds = group_thresholds
    args.dry_run = False
    args.secure_w = False

    args.validation_ratio = validation_ratio

    results_list = []
    for i in range(args.times):
        # print("======== TIME:", i, "start")
        args.seed = args.seed + i
        try:
            sim_results = run_simulation(args, path_project)
            results_list.append(sim_results)
        except OverflowError:
            results_list.append("LOSS IS NAN")
        except AssertionError:
            results_list.append("Assertion Error")

    return results_list


def calc_metric(results, symbol="test"):
    if symbol == "train":
        acc_list = np.array([r['train'][f'train_metric'] for r in results])
        loss_list = np.array([r['train'][f'train_loss'] for r in results])
    else:
        acc_list = np.array([r['global'][f'global_{symbol}'][-1][1] for r in results])
        loss_list = np.array([r['global'][f'global_{symbol}'][-1][2] for r in results])
    acc_mean, acc_std, loss_mean, loss_std = np.mean(acc_list), np.std(acc_list), np.mean(loss_list), np.std(loss_list)
    return acc_mean, acc_std, loss_mean, loss_std


def make_epsilon_u(epsilon=1.0, n_users=0, dist="homo", epsilon_list=[], ratio_list=[], random_state: np.random.RandomState=None) -> dict[int, float]:
    if dist == "homo":
        epsilon_u = {user_id: epsilon for user_id in range(n_users)}
    elif dist == "hetero":
        assert len(epsilon_list) > 0 and len(ratio_list) > 0
        epsilon_u_list = random_state.choice(epsilon_list, size=n_users, p=ratio_list)
        epsilon_u = {user_id: epsilon_u_list[user_id] for user_id in range(n_users)}
    else:
        raise ValueError(f"invalid dist {dist}")
    return epsilon_u

def group_by_closest_below(epsilon_u_dct: dict, group_thresholds: list):
    minimum = min(epsilon_u_dct.values())
    group_thresholds = set(group_thresholds) | {minimum}
    grouped = {
        g: [] for g in group_thresholds
    }  # Initialize the dictionary with empty lists for each group threshold
    for key, value in epsilon_u_dct.items():
        # Find the closest group threshold that is less than or equal to the value
        closest_group = max([g for g in group_thresholds if g <= value], default=None)
        # If a suitable group is found, append the key to the corresponding list
        if closest_group is not None:
            grouped[closest_group].append(key)

    return grouped


## STATIC MANUAL OPTIMIZATION

def prepare_grid_search(epsilon_u, start_idx: int, end_idx: int):
    # set idx list for each group
    group_eps_set = set(epsilon_u.values())
    idx_per_group_list = []

    idx_list = list(range(Q_LIST_SIZE))[start_idx:end_idx]
    idx_list_list = [idx_list for _ in range(len(group_eps_set))]

    for combination in itertools.product(*idx_list_list):
        idx_per_group = {}
        for idx, group_eps in zip(combination, group_eps_set):
            idx_per_group[group_eps] = idx
        idx_per_group_list.append(idx_per_group)

    return {"name": "grid", "params": {"idx_per_group_list": idx_per_group_list}}


def prepare_random_search(epsilon_u, start_idx: int, end_idx: int, random_state: np.random.RandomState, n_samples: int):
    # set idx list for each group``
    group_eps_set = set(epsilon_u.values())
    idx_per_group_list = []

    idx_list = list(range(Q_LIST_SIZE))[start_idx:end_idx]
    idx_list_list = [idx_list for _ in range(len(group_eps_set))]
    all_candidates = itertools.product(*idx_list_list)
    samples = random_state.choice(len(all_candidates), size=n_samples, replace=False)

    for sample in samples:
        idx_per_group = {}
        for idx, group_eps in zip(all_candidates[sample], group_eps_set):
            idx_per_group[group_eps] = idx
        idx_per_group_list.append(idx_per_group)

    return {"name": "random", "params": {"idx_per_group_list": idx_per_group_list}}


def prepare_independent_search(epsilon_u, start_idx: int, end_idx: int):
    # set idx list for each group
    group_eps_set = set(epsilon_u.values())
    idx_per_group_list = []
    idx_list = list(range(Q_LIST_SIZE))[start_idx:end_idx]

    for group_eps in group_eps_set:
        for idx in idx_list:
            idx_per_group = {}
            idx_per_group[group_eps] = idx
            for group_eps in group_eps_set:
                if group_eps != group_eps:
                    idx_per_group[group_eps] = int((start_idx + end_idx) / 2)
        idx_per_group_list.append(idx_per_group)

    return {"name": "independent", "params": {"idx_per_group_list": idx_per_group_list}}


def static_optimization(epsilon_u, sigma, delta, n_users, n_round, dataset_name, times, q_step_size, opt_strategy: dict, global_learning_rate=10.0, local_learning_rate=0.01, local_epochs=30, validation_ratio=0.0):
    try:
        prefix_epsilon_u = list(epsilon_u.items())[:4]
        with open(os.path.join(pickle_path, f'static_optimization_{sigma}_{delta}_{n_users}_{n_round}_{dataset_name}_{q_step_size}_{opt_strategy["name"]}_{validation_ratio}_{prefix_epsilon_u}.pkl'), 'rb') as file:
            results_dict = pickle.load(file)
    except FileNotFoundError:
        results_dict = {}

    if opt_strategy["name"] in ["grid", "random", "independent"]:
        # grid search
        for idx_per_group in opt_strategy["params"]["idx_per_group_list"]:
            print("IDX: ", idx_per_group)
            C_u, q_u = make_static_params(epsilon_u, delta, sigma, n_round, idx_per_group=idx_per_group, q_step_size=q_step_size)
            result = fed_simulation(
                delta, sigma, n_users, C_u=C_u, q_u=q_u, agg_strategy="PULDP-AVG",
                times=times, n_round=n_round, user_dist="zipf-iid", silo_dist="zipf", 
                global_learning_rate=global_learning_rate, local_learning_rate=local_learning_rate, dataset_name=dataset_name,
                local_epochs=local_epochs, epsilon_u=epsilon_u, validation_ratio=validation_ratio,
            )
            results_dict[str(idx_per_group)] = (q_u, C_u, result)
    else:
        raise ValueError(f"invalid opt_strategy {opt_strategy}")
    
    with open(os.path.join(pickle_path, f'static_optimization_{sigma}_{delta}_{n_users}_{n_round}_{dataset_name}_{q_step_size}_{opt_strategy["name"]}_{validation_ratio}_{prefix_epsilon_u}.pkl'), 'wb') as file:
        pickle.dump(results_dict, file)


def show_static_optimization_result(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, opt_strategy: dict, validation_ratio=0.0, train_loss=False):
    prefix_epsilon_u = list(epsilon_u.items())[:4]
    with open(os.path.join(pickle_path, f'static_optimization_{sigma}_{delta}_{n_users}_{n_round}_{dataset_name}_{q_step_size}_{opt_strategy["name"]}_{validation_ratio}_{prefix_epsilon_u}.pkl'), 'rb') as file:
        results_dict = pickle.load(file)

    plt.figure(figsize=(8, 3)) 
    x = list(results_dict.keys())
    q_u_list = [results_dict[i][0][0] for i in x]
    C_u_list = [results_dict[i][1][0] for i in x]
    acc_mean_acc_std_loss_mean_loss_std = [calc_metric(results_dict[i][2], "test") for i in x]
    y = [acc_mean_acc_std_loss_mean_loss_std[i][2] for i in range(len(x))]  # loss_mean
    error = [acc_mean_acc_std_loss_mean_loss_std[i][3] for i in range(len(x))]  # loss_std
    plt.title('Test Loss Mean with Standard Deviation over different idx')
    plt.ylabel('Test Loss Mean')
    plt.errorbar(x, y, yerr=error, fmt='-o')
    for i in range(len(x)):
        plt.text(x[i], y[i]*1.02, f'q={q_u_list[i]:.3f}\nC={C_u_list[i]:.3f}', fontsize=8)
    plt.xlabel('idx')

    ax2 = plt.twinx()
    y_metric = [acc_mean_acc_std_loss_mean_loss_std[i][0] for i in range(len(x))]  # acc_mean
    error_metric = [acc_mean_acc_std_loss_mean_loss_std[i][1] for i in range(len(x))]  # acc_std
    ax2.errorbar(x, y_metric, yerr=error_metric, fmt='-o', color='red', label='Accuracy')
    
    plt.show()

    # optimize for test loss, not for validation loss!
    min_idx, min_test_loss = x[np.argmin(y)], np.min(y)

    if train_loss:
        plt.figure(figsize=(8, 3))
        x = list(results_dict.keys())
        q_u_list = [results_dict[i][0][0] for i in x]
        C_u_list = [results_dict[i][1][0] for i in x]
        acc_mean_acc_std_loss_mean_loss_std = [calc_metric(results_dict[i][2], "train") for i in x]
        y = [acc_mean_acc_std_loss_mean_loss_std[i][2] for i in range(len(x))]  # loss_mean
        error = [acc_mean_acc_std_loss_mean_loss_std[i][3] for i in range(len(x))]  # loss_std
        plt.errorbar(x, y, yerr=error, fmt='-o')
        for i in range(len(x)):
            plt.text(x[i], y[i]*1.02, f'q={q_u_list[i]:.3f}\nC={C_u_list[i]:.3f}', fontsize=8)
        plt.title('Train Loss Mean with Standard Deviation over different idx')
        plt.xlabel('idx')
        plt.ylabel('Train Loss Mean')

        ax2 = plt.twinx()
        y_metric = [acc_mean_acc_std_loss_mean_loss_std[i][0] for i in range(len(x))]  # acc_mean
        error_metric = [acc_mean_acc_std_loss_mean_loss_std[i][1] for i in range(len(x))]  # acc_std
        ax2.errorbar(x, y_metric, yerr=error_metric, fmt='-o', color='red', label='Accuracy')

        plt.show()
        min_idx, min_test_loss = x[np.argmin(y)], np.min(y)

    if validation_ratio > 0.0:
        plt.figure(figsize=(8, 3)) 
        x = list(results_dict.keys())
        q_u_list = [results_dict[i][0][0] for i in x]
        C_u_list = [results_dict[i][1][0] for i in x]
        acc_mean_acc_std_loss_mean_loss_std = [calc_metric(results_dict[i][2], "valid") for i in x]
        y = [acc_mean_acc_std_loss_mean_loss_std[i][2] for i in range(len(x))]  # loss_mean
        error = [acc_mean_acc_std_loss_mean_loss_std[i][3] for i in range(len(x))]  # loss_std
        plt.errorbar(x, y, yerr=error, fmt='-o')
        for i in range(len(x)):
            plt.text(x[i], y[i]*1.02, f'q={q_u_list[i]:.3f}\nC={C_u_list[i]:.3f}', fontsize=8)
        plt.title('Valid Loss Mean with Standard Deviation over different idx')
        plt.xlabel('idx')
        plt.ylabel('Valid Loss Mean')

        ax2 = plt.twinx()
        y_metric = [acc_mean_acc_std_loss_mean_loss_std[i][0] for i in range(len(x))]  # acc_mean
        error_metric = [acc_mean_acc_std_loss_mean_loss_std[i][1] for i in range(len(x))]  # acc_std
        ax2.errorbar(x, y_metric, yerr=error_metric, fmt='-o', color='red', label='Accuracy')

        plt.show()

    return min_idx, min_test_loss


def run_with_specified_idx(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, times, idx_per_group, global_learning_rate=10.0, local_learning_rate=0.01, local_epochs=30, validation_ratio=0.0):
    C_u, q_u = make_static_params(epsilon_u, delta, sigma, n_round, idx_per_group=idx_per_group, q_step_size=q_step_size)
    result = fed_simulation(
        delta, sigma, n_users, C_u=C_u, q_u=q_u, agg_strategy="PULDP-AVG",
        times=times, n_round=n_round, user_dist="zipf-iid", silo_dist="zipf", 
        global_learning_rate=global_learning_rate, local_learning_rate=local_learning_rate, dataset_name=dataset_name,
        local_epochs=local_epochs, epsilon_u=epsilon_u, validation_ratio=validation_ratio,
    )

    acc_mean, acc_std, loss_mean, loss_std = calc_metric(result, "test")
    print(f"TEST ACC: {acc_mean:.4f} ± {acc_std:.4f}", f", TEST LOSS: {loss_mean:.4f} ± {loss_std:.4f}")

    if validation_ratio > 0.0:
        acc_mean, acc_std, loss_mean, loss_std = calc_metric(result, "valid")
        print(f"VALID ACC: {acc_mean:.4f} ± {acc_std:.4f}", f", VALID LOSS: {loss_mean:.4f} ± {loss_std:.4f}")

    prefix_epsilon_u = list(epsilon_u.items())[:4]
    with open(os.path.join(pickle_path, f'specified_idx_{n_users}_{sigma}_{delta}_{dataset_name}_{n_round}_{idx_per_group}_{q_step_size}_{validation_ratio}_{prefix_epsilon_u}.pkl'), 'wb') as file:
        pickle.dump(result, file)


## ONLINE OPTIMIZATION

def run_online_optimization(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, group_thresholds, times, global_learning_rate=10.0, local_learning_rate=0.01, local_epochs=30, validation_ratio=0.0):
    result = fed_simulation(
        delta, sigma, n_users, C_u=None, q_u=None, q_step_size=q_step_size, agg_strategy="PULDP-AVG-online",
        times=times, n_round=n_round, user_dist="zipf-iid", silo_dist="zipf", 
        global_learning_rate=global_learning_rate, local_learning_rate=local_learning_rate, dataset_name=dataset_name,
        local_epochs=local_epochs,
        epsilon_u=epsilon_u,
        group_thresholds=group_thresholds, validation_ratio=validation_ratio,
    )

    acc_mean, acc_std, loss_mean, loss_std = calc_metric(result)
    print(f"TEST ACC: {acc_mean:.4f} ± {acc_std:.4f}", f", TEST LOSS: {loss_mean:.4f} ± {loss_std:.4f}")

    if validation_ratio > 0.0:
        acc_mean, acc_std, loss_mean, loss_std = calc_metric(result, "valid")
        print(f"VALID ACC: {acc_mean:.4f} ± {acc_std:.4f}", f", VALID LOSS: {loss_mean:.4f} ± {loss_std:.4f}")

    prefix_epsilon_u = list(epsilon_u.items())[:4]
    with open(os.path.join(pickle_path, f'result_{n_users}_{sigma}_{delta}_{dataset_name}_{n_round}_{q_step_size}_{validation_ratio}_{prefix_epsilon_u}.pkl'), 'wb') as file:
        pickle.dump(result, file)


def show_online_optimization_result(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, validation_ratio=0.0):
    with open(os.path.join(pickle_path, f'result_{n_users}_{sigma}_{epsilon_list}_{group_thresholds}_{ratio_list}_{delta}_{dataset_name}_{n_round}_{q_step_size}_{validation_ratio}.pkl'), 'rb') as file:
        result = pickle.load(file)

    # eps_uの値のリストを取得（すべての辞書から共通のキーを抽出）
    eps_u_values = set(key for dct in result for key in dct["param_history"].keys())

    fig, ax1 = plt.subplots()
    for eps_u in eps_u_values:
        # 各辞書からeps_uに対応するデータを集める
        all_data = np.array([dct["param_history"][eps_u][:-1] for dct in result if eps_u in dct["param_history"]])

        # 平均値と標準偏差を計算
        means = np.mean(all_data, axis=0)
        stds = np.std(all_data, axis=0)

        # 平均値と標準偏差をプロット
        x = range(len(means))
        y1 = [item[0] for item in means]
        y2 = [item[1] for item in means]
        error1 = [item[0] for item in stds]
        error2 = [item[1] for item in stds]

        ax1.errorbar(x, y1, yerr=error1, label=f'eps_u={eps_u} q_u', alpha=0.5)
        ax1.errorbar(x, y2, yerr=error2, label=f'eps_u={eps_u} C_u', alpha=0.5)

    loss_means = []
    loss_stds = []
    acc_means = []
    acc_stds = []

    # 各ラウンドに対して処理
    for i in range(len(result[0]['global']['global_test'])):
        # その位置における全ラウンドのloss値を集める
        losses_at_position = [result[round_id]['global']['global_test'][i][2] for round_id in range(len(result))]
        accs_at_position = [result[round_id]['global']['global_test'][i][1] for round_id in range(len(result))]

        loss_means.append(np.mean(losses_at_position))
        loss_stds.append(np.std(losses_at_position))

        acc_means.append(np.mean(accs_at_position))
        acc_stds.append(np.std(accs_at_position))

    ax2 = ax1.twinx()
    ax2.errorbar(x, loss_means, yerr=loss_stds, label=f'Test Loss', color='red', alpha=0.5)
    ax1.set_yscale('log')
    ax1.set_xlabel('Round')
    ax1.set_ylabel('q_u and C_u')
    ax2.set_ylabel('Test Loss')
    ax2.set_yscale('log')
    plt.title('q_u and C_u with Test Loss')
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    # グラフの表示
    plt.show()

    fig, ax = plt.subplots()
    ax.errorbar(x, acc_means, yerr=acc_stds, label=f'Test Accuracy', color='red', alpha=0.5)
    ax.set_xlabel('Round')
    ax.set_ylabel('Test Accuracy')
    ax.set_title('Test Accuracy')
    ax.legend(loc='upper left')
    plt.show()


def show_online_optimization_result_from_n_users_list(n_users_list, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, optimal_dct=None):
    _, ax_q = plt.subplots()
    _, ax_loss = plt.subplots()
    for n_users in n_users_list:
        with open(os.path.join(pickle_path, f'result_{n_users}_{sigma}_{epsilon_list}_{group_thresholds}_{ratio_list}_{delta}_{dataset_name}_{n_round}_{q_step_size}.pkl'), 'rb') as file:
            result = pickle.load(file)

        # eps_uの値のリストを取得（すべての辞書から共通のキーを抽出）
        eps_u_values = set(key for dct in result for key in dct["param_history"].keys())

        for eps_u in eps_u_values:
            # 各辞書からeps_uに対応するデータを集める
            all_data = np.array([dct["param_history"][eps_u][:-1] for dct in result if eps_u in dct["param_history"]])

            # 平均値と標準偏差を計算
            means = np.mean(all_data, axis=0)
            stds = np.std(all_data, axis=0)

            # 平均値と標準偏差をプロット
            x = range(len(means))
            y1 = [item[0] for item in means]
            error1 = [item[0] for item in stds]
            ax_q.errorbar(x, y1, yerr=error1, label=f'q_u eps_u={eps_u} n_users={n_users}', alpha=0.9)

        loss_means = []
        loss_stds = []
        acc_means = []
        acc_stds = []

        # 各ラウンドに対して処理
        for i in range(len(result[0]['global']['global_test'])):
            # その位置における全ラウンドのloss値を集める
            losses_at_position = [result[round_id]['global']['global_test'][i][2] for round_id in range(len(result))]
            accs_at_position = [result[round_id]['global']['global_test'][i][1] for round_id in range(len(result))]

            loss_means.append(np.mean(losses_at_position))
            loss_stds.append(np.std(losses_at_position))

            acc_means.append(np.mean(accs_at_position))
            acc_stds.append(np.std(accs_at_position))
    
        ax_loss.errorbar(x, loss_means, yerr=loss_stds, label=f'Test Loss n_users={n_users}', alpha=0.5)

    
    # optimal q_u
    if optimal_dct is not None and type(optimal_dct) is dict:
        for (eps_u, n_users, idx), q_u in optimal_dct.items():
            with open(os.path.join(pickle_path, f'result_{n_users}_{sigma}_{epsilon_list}_{group_thresholds}_{ratio_list}_{delta}_{dataset_name}_{n_round}_{idx}_{q_step_size}.pkl'), 'rb') as file:
                result = pickle.load(file)

            loss_means = []
            loss_stds = []
            acc_means = []
            acc_stds = []

            for i in range(len(result[0]['global']['global_test'])):
                losses_at_position = [result[round_id]['global']['global_test'][i][2] for round_id in range(len(result))]
                accs_at_position = [result[round_id]['global']['global_test'][i][1] for round_id in range(len(result))]

                loss_means.append(np.mean(losses_at_position))
                loss_stds.append(np.std(losses_at_position))

                acc_means.append(np.mean(accs_at_position))
                acc_stds.append(np.std(accs_at_position))
            
            ax_loss.errorbar(x, loss_means, yerr=loss_stds, label=f'Test Loss optimal q C eps_u={eps_u} n_users={n_users}', alpha=0.5)
            ax_q.errorbar(x, [q_u]*len(x), yerr=[0.0]*len(x), label=f'Optimal q_u eps_u={eps_u} n_users={n_users}', alpha=0.9)

    ax_q.set_yscale('log')
    ax_q.set_xlabel('Round')
    ax_q.set_ylabel('q_u')
    ax_q.legend(loc='lower left')

    ax_loss.set_ylabel('Test Loss')
    ax_loss.set_yscale('log')
    ax_loss.legend(loc='upper right')

    # グラフの表示
    plt.show()


def compare_RDP_with_different_trials(sample_rate=0.1, sigma=5.0, t=100, n_eps_groups=10):
    # sub-sampled Gaussian mechanismのRDPにおいて，定数倍の試行回数がどのような影響を与えるかを調べる
    accountant = RDPAccountant()
    for i in range(t):
        # accountant.compose_subsampled_mechanisms_lowerbound(func=func_gaussian, prob=q_u)
        accountant.step(noise_multiplier=sigma, sample_rate=sample_rate)

    print(f'(t={t}) epsilon={accountant.get_epsilon(delta=1e-5)}')

    t = t * n_eps_groups
    accountant = RDPAccountant()
    for i in range(t):
        # accountant.compose_subsampled_mechanisms_lowerbound(func=func_gaussian, prob=q_u)
        accountant.step(noise_multiplier=sigma, sample_rate=sample_rate)

    print(f'(t={t}) epsilon={accountant.get_epsilon(delta=1e-5)}')

# Check monotonically increasing between q and C for epsilon

In [ ]:
delta = 1e-5
sigma = 5.0
epsilon_u = 5.0
T = 1000

q_u_list = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.8, 0.99]
sensitivity_u_list = [0.01, 0.1, 1.0, 5.0, 10, 100]
data = []

for q_u in q_u_list:
    for sensitivity_u in sensitivity_u_list:
        func_gaussian = lambda x: RDP_gaussian_with_C(sigma, x, sensitivity_u)
        accountant = rdp_acct.anaRDPacct(m=100)
        for i in range(T):
            accountant.compose_subsampled_mechanisms_lowerbound(func=func_gaussian, prob=q_u)
        eps = accountant.get_eps(delta)
        if eps < 1e10:
            data.append((q_u, sensitivity_u, eps))
        else:
            continue


# for q_u in q_u_list
fig = plt.figure(figsize=(15, 2))
for i, q_u in enumerate(q_u_list):
    filtered_data = [d for d in data if d[0] == q_u]
    x = [d[1] for d in filtered_data]
    y = [d[2] for d in filtered_data]
    ax = fig.add_subplot(1, len(q_u_list), i+1)
    ax.plot(x, y, label='q_u = {}'.format(q_u), marker='o')
    ax.set_title('q_u = {}'.format(q_u), size=14)
    ax.set_xlabel('sensitivity_u', size=14)
    if i == 0:
        ax.set_ylabel('epsilon', size=14)
    ax.set_xscale('log')
    ax.set_yscale('log')
    # ax.set_ylim(0, epsilon_u+1.0)
plt.show()

# for sensitivity_u in sensitivity_u_list
fig = plt.figure(figsize=(18, 2))
for i, sensitivity_u in enumerate(sensitivity_u_list):
    filtered_data = [d for d in data if d[1] == sensitivity_u]
    x = [d[0] for d in filtered_data]
    y = [d[2] for d in filtered_data]
    ax = fig.add_subplot(1, len(sensitivity_u_list), i+1)
    ax.plot(x, y, label='sensitivity_u = {}'.format(sensitivity_u), marker='o')
    ax.set_title('sensitivity_u = {}'.format(sensitivity_u), size=14)
    ax.set_xlabel('q_u', size=14)
    if i == 0:
        ax.set_ylabel('epsilon', size=14)
    ax.set_xscale('log')
    ax.set_yscale('log')
    # ax.set_ylim(0, epsilon_u+1.0)
plt.show()

# Binary Search based method

- `accountant = rdp_acct.anaRDPacct(m=m)`
    - Zhu, Yuqing, and Yu-Xiang Wang. "Poission subsampled rényi differential privacy." International Conference on Machine Learning. PMLR, 2019.
- `accountant = RDPAccountant()`
    - Mironov, Ilya, Kunal Talwar, and Li Zhang. "R\'enyi differential privacy of the sampled gaussian mechanism." arXiv preprint arXiv:1908.10530 (2019).

In [ ]:
delta = 1e-5
sigma = 5.0
epsilon_u = 5.0
T = 100

sensitivity_u, eps = from_q_u(q_u=0.1, delta=delta, epsilon_u=epsilon_u, sigma=sigma, T=T)
print("sensitivity_u =", sensitivity_u, "eps =", eps)

q_u, eps = from_sensitivity_u(sensitivity_u=1.59, delta=delta, epsilon_u=epsilon_u, sigma=sigma, T=T)
print("q_u =", q_u, "eps =", eps)

# Observation
- to observe what happens with various sensitivity and sampling rate

Parameters
- epsilon_u, delta_u
    - privacy budgets for each users
- homo, hetero
    - distribution of privacy budgets for each users

### Plotting the pairs

In [ ]:
epsilon_u = 3.0
n_round = 20
delta = 1e-5
for sigma in [0.5]:
    x, y = make_q_c_curve(epsilon_u=epsilon_u, delta=delta, sigma=sigma, num_points=30, n_round=n_round, min=-6)
    plot_q_c_curve(x, y, title=f"n_round = {n_round}, epsilon_u = {epsilon_u}, delta = {delta}, sigma = {sigma}")

In [ ]:
epsilon_u = 3.0
n_round = 10
delta = 1e-5
for sigma in [0.5]:
    x, y = make_q_c_curve(epsilon_u=epsilon_u, delta=delta, sigma=sigma, num_points=30, n_round=n_round, min=-6)
    plot_q_c_curve(x, y, title=f"n_round = {n_round}, epsilon_u = {epsilon_u}, delta = {delta}, sigma = {sigma}", log=True)

In [ ]:
delta = 1e-5
# epsilon_u = 1.0
# sigma = 0.5
# for n_round in [10, 100, 1000]:
#     x, y = make_q_c_curve(epsilon_u=epsilon_u, delta=delta, sigma=0.5, num_points=30, n_round=n_round, min=-6)
#     plot_q_c_curve(x, y, title=f"n_round = {n_round}, epsilon_u = {epsilon_u}, delta = {delta}, sigma = {sigma}")

# epsilon_u = 10.0
# n_round = 100
# for n_round in [10, 100, 1000]:
#     x, y = make_q_c_curve(epsilon_u=epsilon_u, delta=delta, sigma=0.5, num_points=30, n_round=n_round, min=-6)
#     plot_q_c_curve(x, y, title=f"n_round = {n_round}, epsilon_u = {epsilon_u}, delta = {delta}, sigma = {sigma}")


epsilon_u = 3.0
n_round = 20
sigma = 0.5
for epsilon_u in [0.11, 1.0, 5.0, 10.0]:
    x, y = make_q_c_curve(epsilon_u=epsilon_u, delta=delta, sigma=sigma, num_points=30, n_round=n_round, min=-6)
    plot_q_c_curve(x, y, title=f"n_round = {n_round}, epsilon_u = {epsilon_u}, delta = {delta}, sigma = {sigma}")
    plot_q_c_curve(x, y, title=f"n_round = {n_round}, epsilon_u = {epsilon_u}, delta = {delta}, sigma = {sigma}", log=False)


# Machine Learning

## Manual HP search

### The relationship between step size and q_u

In [ ]:
# 各ステップサイズに対するq_listを計算
step_size = [0.8, 0.9]
all_q_lists = []
for step in step_size:
    q_list = []
    init_q = 1.0
    for _ in range(30):
        q_list.append(init_q)
        init_q *= step
    all_q_lists.append(q_list)

plt.figure(figsize=(7, 4))
for i, q_list in enumerate(all_q_lists):
    plt.plot(q_list, label=f'Step Size {step_size[i]}', marker='o')
plt.yscale('log')
plt.title('Q Values Over Time with Different Step Sizes (Log Scale)')
plt.xlabel('Iteration')
plt.ylabel('Q Value')
plt.legend()
plt.show()

### heart disease

In [ ]:
sigma = 1.0
epsilon_list = [0.5]
group_thresholds = [0.5]
ratio_list = [1.0]
delta = 1e-5
n_users = 100
n_round = 30
dataset_name = 'heart_disease'
q_step_size = 0.8
times = 5

random_state = np.random.RandomState(0)
epsilon_u = make_epsilon_u(n_users=n_users, dist="hetero", epsilon_list=epsilon_list, ratio_list=ratio_list, random_state=random_state)
grouped = group_by_closest_below(epsilon_u_dct=epsilon_u, group_thresholds=group_thresholds)
epsilon_u = {}
for eps_u, user_ids in grouped.items():
    for user_id in user_ids:
        epsilon_u[user_id] = eps_u
opt_strategy = prepare_grid_search(epsilon_u, start_idx=0, end_idx=12)

logger_set_warning()
# static_optimization(epsilon_u, sigma, delta, n_users, n_round, dataset_name, times, q_step_size, opt_strategy=opt_strategy, global_learning_rate=10.0, local_learning_rate=0.001, local_epochs=50, validation_ratio=0.9)
min_idx, min_loss = show_static_optimization_result(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, opt_strategy=opt_strategy, validation_ratio=0.9, train_loss=True)
print(min_idx, min_loss)

In [ ]:
sigma = 1.0
epsilon_list = [0.5]
group_thresholds = [0.5]
ratio_list = [1.0]
delta = 1e-5
n_users = 300
n_round = 30
dataset_name = 'heart_disease'
q_step_size = 0.8
times = 5

random_state = np.random.RandomState(0)
epsilon_u = make_epsilon_u(n_users=n_users, dist="hetero", epsilon_list=epsilon_list, ratio_list=ratio_list, random_state=random_state)
grouped = group_by_closest_below(epsilon_u_dct=epsilon_u, group_thresholds=group_thresholds)
epsilon_u = {}
for eps_u, user_ids in grouped.items():
    for user_id in user_ids:
        epsilon_u[user_id] = eps_u
opt_strategy = prepare_grid_search(epsilon_u, start_idx=0, end_idx=12)

logger_set_warning()
# static_optimization(epsilon_u, sigma, delta, n_users, n_round, dataset_name, times, q_step_size, opt_strategy=opt_strategy, global_learning_rate=10.0, local_learning_rate=0.001, local_epochs=50, validation_ratio=0.9)
min_idx, min_loss = show_static_optimization_result(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, opt_strategy=opt_strategy, validation_ratio=0.9, train_loss=True)
print(min_idx, min_loss)

### tcga brca

In [4]:
sigma = 1.0
epsilon_list = [1.0]
group_thresholds = [1.0]
ratio_list = [1.0]
delta = 1e-5
n_users = 100
n_round = 1
dataset_name = 'tcga_brca'
q_step_size = 0.8
times = 5

random_state = np.random.RandomState(0)
epsilon_u = make_epsilon_u(n_users=n_users, dist="hetero", epsilon_list=epsilon_list, ratio_list=ratio_list, random_state=random_state)
grouped = group_by_closest_below(epsilon_u_dct=epsilon_u, group_thresholds=group_thresholds)
epsilon_u = {}
for eps_u, user_ids in grouped.items():
    for user_id in user_ids:
        epsilon_u[user_id] = eps_u
opt_strategy = prepare_grid_search(epsilon_u, start_idx=0, end_idx=12)

logger_set_warning()
static_optimization(epsilon_u, sigma, delta, n_users, n_round, dataset_name, times, q_step_size, opt_strategy=opt_strategy, global_learning_rate=10.0, local_learning_rate=0.001, local_epochs=50, validation_ratio=0.9)
min_idx, min_loss = show_static_optimization_result(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, opt_strategy=opt_strategy, validation_ratio=0.9)
print(min_idx, min_loss)

IDX:  {1.0: 0}
user 72
[[0.000e+00 7.950e+02]
 [0.000e+00 1.648e+03]
 [0.000e+00 1.417e+03]
 [0.000e+00 6.290e+02]
 [0.000e+00 2.403e+03]
 [1.000e+00 1.009e+03]
 [0.000e+00 3.456e+03]]
[[-0.27548307]
 [-0.5111072 ]
 [-0.51046723]
 [-0.32073528]
 [-0.7735959 ]
 [ 0.01045175]
 [-0.2810343 ]]
user 74
[[   0. 2541.]
 [   0. 2109.]
 [   0.  829.]
 [   0. 1132.]
 [   0. 2477.]
 [   0. 1842.]
 [   0. 2653.]]
[[-0.36485785]
 [-0.46148992]
 [-0.58896226]
 [-0.02240522]
 [-0.65942127]
 [-0.4768992 ]
 [-0.30266982]]
user 78
[[   0.  567.]
 [   0. 2311.]
 [   0.  216.]
 [   0.  287.]]
[[-0.30634898]
 [-0.48187053]
 [-0.00408075]
 [ 0.34192538]]
user 53
[[0.000e+00 1.613e+03]
 [0.000e+00 2.064e+03]
 [0.000e+00 1.434e+03]
 [1.000e+00 5.630e+02]
 [0.000e+00 2.255e+03]
 [0.000e+00 1.528e+03]
 [0.000e+00 6.200e+02]
 [1.000e+00 1.148e+03]]
[[ 0.09959295]
 [-0.4380104 ]
 [-0.29307896]
 [-0.39281833]
 [ 0.00616425]
 [-0.45016724]
 [-0.7527606 ]
 [-0.3882832 ]]
user 60
[[0.000e+00 7.590e+02]
 [0.000e+00 1.

KeyboardInterrupt: 

In [ ]:
sigma = 1.0
epsilon_list = [1.0]
group_thresholds = [1.0]
ratio_list = [1.0]
delta = 1e-5
n_users = 130
n_round = 30
dataset_name = 'tcga_brca'
q_step_size = 0.8
times = 5

random_state = np.random.RandomState(0)
epsilon_u = make_epsilon_u(n_users=n_users, dist="hetero", epsilon_list=epsilon_list, ratio_list=ratio_list, random_state=random_state)
grouped = group_by_closest_below(epsilon_u_dct=epsilon_u, group_thresholds=group_thresholds)
epsilon_u = {}
for eps_u, user_ids in grouped.items():
    for user_id in user_ids:
        epsilon_u[user_id] = eps_u
opt_strategy = prepare_grid_search(epsilon_u, start_idx=0, end_idx=12)

static_optimization(epsilon_u, sigma, delta, n_users, n_round, dataset_name, times, q_step_size, opt_strategy=opt_strategy, global_learning_rate=10.0, local_learning_rate=0.001, local_epochs=50, validation_ratio=0.9)
min_idx, min_loss = show_static_optimization_result(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, opt_strategy=opt_strategy, validation_ratio=0.9, train_loss=True)
print(min_idx, min_loss)

### creditcard

In [ ]:
sigma = 0.5
epsilon_list = [3.0]
group_thresholds = [3.0]
ratio_list = [1.0]
delta = 1e-5
n_users = 300
n_round = 30
dataset_name = 'creditcard'
q_step_size=0.8
times = 5

random_state = np.random.RandomState(0)
epsilon_u = make_epsilon_u(n_users=n_users, dist="hetero", epsilon_list=epsilon_list, ratio_list=ratio_list, random_state=random_state)
grouped = group_by_closest_below(epsilon_u_dct=epsilon_u, group_thresholds=group_thresholds)
epsilon_u = {}
for eps_u, user_ids in grouped.items():
    for user_id in user_ids:
        epsilon_u[user_id] = eps_u
opt_strategy = prepare_grid_search(epsilon_u, start_idx=0, end_idx=12)

logger_set_warning()
static_optimization(epsilon_u, sigma, delta, n_users, n_round, dataset_name, times, q_step_size, opt_strategy=opt_strategy, global_learning_rate=10.0, local_learning_rate=0.01, local_epochs=30, validation_ratio=0.5)
min_idx, min_loss = show_static_optimization_result(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, opt_strategy=opt_strategy, validation_ratio=0.9, train_loss=True)


In [ ]:
sigma = 0.5
epsilon_list = [5.0]
group_thresholds = [5.0]
ratio_list = [1.0]
delta = 1e-5
n_users = 1000
n_round = 30
dataset_name = 'creditcard'
q_step_size=0.8
times = 5

random_state = np.random.RandomState(0)
epsilon_u = make_epsilon_u(n_users=n_users, dist="hetero", epsilon_list=epsilon_list, ratio_list=ratio_list, random_state=random_state)
grouped = group_by_closest_below(epsilon_u_dct=epsilon_u, group_thresholds=group_thresholds)
epsilon_u = {}
for eps_u, user_ids in grouped.items():
    for user_id in user_ids:
        epsilon_u[user_id] = eps_u
opt_strategy = prepare_grid_search(epsilon_u, start_idx=0, end_idx=12)

static_optimization(epsilon_u, sigma, delta, n_users, n_round, dataset_name, times, q_step_size, opt_strategy=opt_strategy, global_learning_rate=10.0, local_learning_rate=0.01, local_epochs=30, validation_ratio=0.5)
min_idx, min_loss = show_static_optimization_result(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, opt_strategy=opt_strategy, validation_ratio=0.9, train_loss=True)


### run with specified (best) idx

In [ ]:
n_users = 300
sigma = 1.0
epsilon_list = [3.0]
group_thresholds = [3.0]
ratio_list = [1.0]
delta = 1e-5
dataset_name = 'heart_disease'
n_round=30
idx_per_group = {3.0: 4}
q_step_size = 0.8
times = 5

random_state = np.random.RandomState(0)
epsilon_u = make_epsilon_u(n_users=n_users, dist="hetero", epsilon_list=epsilon_list, ratio_list=ratio_list, random_state=random_state)
grouped = group_by_closest_below(epsilon_u_dct=epsilon_u, group_thresholds=group_thresholds)
epsilon_u = {}
for eps_u, user_ids in grouped.items():
    for user_id in user_ids:
        epsilon_u[user_id] = eps_u
opt_strategy = prepare_grid_search(epsilon_u, start_idx=0, end_idx=12)

run_with_specified_idx(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, times, idx_per_group, global_learning_rate=10.0, local_learning_rate=0.001, local_epochs=50, validation_ratio=0.9)


In [ ]:
n_users = 100
sigma = 0.5
epsilon_list = [3.0]
group_thresholds = [3.0]
ratio_list = [1.0]
delta = 1e-5
dataset_name = 'tcga_brca'
n_round = 30
idx_per_group = {3.0: 2}
q_step_size = 0.8
times = 5

random_state = np.random.RandomState(0)
epsilon_u = make_epsilon_u(n_users=n_users, dist="hetero", epsilon_list=epsilon_list, ratio_list=ratio_list, random_state=random_state)
grouped = group_by_closest_below(epsilon_u_dct=epsilon_u, group_thresholds=group_thresholds)
epsilon_u = {}
for eps_u, user_ids in grouped.items():
    for user_id in user_ids:
        epsilon_u[user_id] = eps_u
opt_strategy = prepare_grid_search(epsilon_u, start_idx=0, end_idx=12)

run_with_specified_idx(epsilon_u, sigma, delta, n_users, n_round, dataset_name, q_step_size, times, idx_per_group, global_learning_rate=10.0, local_learning_rate=0.001, local_epochs=50, validation_ratio=0.9)


## Single Epsilon group

In [ ]:
# heart disease with different delta
n_users_list = [100, 200, 300]
sigma = 1.0
epsilon_list = [3.0]
group_thresholds = [3.0]
ratio_list = [1.0]
delta = 1e-5
dataset_name = 'heart_disease'
n_round = 30
q_step_size = 0.8
times = 5

for n_users in n_users_list:
    print("n_users:", n_users)
    run_online_optimization(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, times, global_learning_rate=1.0, local_epochs=50)
    show_online_optimization_result(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size)

show_online_optimization_result_from_n_users_list(n_users_list, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, optimal_dct={(3.0, 300, 4): 0.41})


In [ ]:
# herat disease with different delta
n_users_list = [100]
sigma = 0.5
epsilon_list = [3.0]
group_thresholds = [3.0]
ratio_list = [1.0]
delta = 1e-5
dataset_name = 'tcga_brca'
n_round = 30
q_step_size = 0.8
times = 5

for n_users in n_users_list:
    print("n_users:", n_users)
    # run_online_optimization(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, times, global_learning_rate=1.0, local_epochs=50)
    # show_online_optimization_result(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size)
show_online_optimization_result_from_n_users_list(n_users_list, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, optimal_dct={(3.0, 100, 2): 0.64})


In [ ]:
# creditcard with different number of users
n_users_list = [100, 1000, 2000]
global_learning_rate_list = [10.0, 20.0, 20.0]

sigma = 0.5
epsilon_list = [10.0]
group_thresholds = [10.0]
ratio_list = [1.0]
delta = 1e-5
dataset_name = 'creditcard'
n_round=50
times = 3
q_step_size = 0.8

for n_users, global_learning_rate in zip(n_users_list, global_learning_rate_list):
    print("n_users:", n_users)
    run_online_optimization(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, times, global_learning_rate=10.0)
    show_online_optimization_result(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size)


In [ ]:
# creditcard with different number of users
n_users_list = [2000]
global_learning_rate_list = [10.0]

sigma = 1.0
epsilon_list = [5.0]
group_thresholds = [5.0]
ratio_list = [1.0]
delta = 1e-5
dataset_name = 'creditcard'
n_round = 50
times = 5
q_step_size = 0.8

for n_users, global_learning_rate in zip(n_users_list, global_learning_rate_list):
    print("n_users:", n_users)
    run_online_optimization(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, times, global_learning_rate=global_learning_rate)
    show_online_optimization_result(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size)


In [ ]:
# mnist with different number of users
n_users_list = [100, 1000]
global_learning_rate_list = [5.0, 10.0]

sigma = 0.5
epsilon_list = [5.0]
group_thresholds = [5.0]
ratio_list = [1.0]
delta = 1e-5
dataset_name = 'mnist'
n_round = 30
times = 3

for n_users, global_learning_rate in zip(n_users_list, global_learning_rate_list):
    print("n_users:", n_users)
    run_online_optimization(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, times, global_learning_rate=global_learning_rate)


## Multiple Epsilon groups

In [ ]:
n_users = 300
sigma = 5.0
epsilon_list = [1.0, 5.0, 10.0]
group_thresholds = epsilon_list
ratio_list = [0.3, 0.4, 0.3]
delta = 1e-5
dataset_name = 'heart_disease'
n_round=20
q_step_size = 0.8
global_learning_rate = 10
times = 5

run_online_optimization(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size, times, global_learning_rate=10.0, local_learning_rate=0.001, local_epochs=30)
show_online_optimization_result(n_users, sigma, epsilon_list, group_thresholds, ratio_list, delta, dataset_name, n_round, q_step_size)


In [ ]:
epsilon_list = [1.0, 5.0, 10.0]
user_group_size = len(epsilon_list)
ratio_list = [0.3, 0.4, 0.3]
idx_per_group = {1.0: 4, 5.0: 4, 10.0: 4}
n_users = 300
delta = 0.00001
sigma = 0.5
n_round = 20
idx = 4
q_step_size = 0.8

epsilon_u = make_epsilon_u(n_users=n_users, dist="hetero", epsilon_list=epsilon_list, ratio_list=ratio_list)
C_u, q_u = make_static_params(epsilon_u, delta, sigma, n_round, idx_per_group=idx_per_group, q_step_size=q_step_size)